In [ ]:
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error, mean_absolute_error
import time
import pandas as pd
import numpy as np
import warnings
from itertools import combinations
import requests
import seaborn as sns
from matplotlib import pyplot as plt
import pickle
from pandas.api.types import CategoricalDtype
import warnings

warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [ ]:
columns_name = ["age", "workClass", "fnlwgt", "education", "education.num",
                "marital.status", "occupation", "relationship", "race", "sex", 
                "capital_gain", "capital_loss", "hours_perweek", "native_country", "income"]

train_df = pd.read_csv('../data/adult.data', names=columns_name, sep=' *, *')
test_df = pd.read_csv('../data/adult.test',  names=columns_name, sep=' *, *', skiprows=1)

# ----------------------------preprocessing--------------------------------

1. concate train_df and test_df

In [ ]:
test_X = test_df.drop('income', axis=1)
df = pd.concat([train_df, test_X], ignore_index=True)

2. Explaced formate error "?" to be None

In [ ]:
df[df == '?'] = np.nan

3. The dependent column ‘income’ which is to be predicted has been replaced with 0 and 1 and hence convert the problem to a dichotomo

In [ ]:
df['income'].replace({'<=50K':0,'>50K':1}, inplace=True)
Test_y = test_df['income']
Test_y.replace({'<=50K.':0, '>50K.':1}, inplace=True)

4. Delete useless features

In [ ]:
df = df.drop('fnlwgt',axis=1)
df = df.drop('education.num', axis=1)

# ----------------------------------Feature engineer--------------------------------------

Writen a custom transformer which will select the corresponding attributes

In [1]:
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import FeatureUnion
from sklearn.model_selection import cross_val_score

In [ ]:
class featureSelector(BaseEstimator, TransformerMixin):
  
    def __init__(self, type):
        self.type = type
  
    def fit(self, X, y=None):
        return self

    def transform(self,X):
        return X.select_dtypes(include=[self.type])

Developed the numerical Data Pipeline

In [ ]:
class array_df(BaseEstimator, TransformerMixin):
    
    def _init_(self, name=['age', 'capital-gain', 'capital-loss', 'hours.per.week']):
        self.names = name
        
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        d = pd.DataFrame(X, columns = ['age', 'capital-gain', 'capital-loss', 'hours.per.week'])
        return d

In [ ]:
num_pipeline = Pipeline(steps=[
    ("num_selector", featureSelector(type='int')),
    ("scaler", StandardScaler()),
    ('arrayToDf', array_df())
])

Complete Numerical Pipeline

In [ ]:
num_pipeline = Pipeline(steps=[
    ("num_selector", featureSelector(type='int')),
    ("scaler", StandardScaler()),
    ('arrayToDf', array_df())
])

Developed the Categorical Data Pipeline

Replacing the missing values with the most frequently occurring value in each column in the categorical columns.

In [ ]:
class CategoricalImputer(BaseEstimator, TransformerMixin):
  
    def __init__(self, columns = None, strategy='most_frequent'):
        self.strategy = strategy
        self.columns = columns
    
    
    def fit(self,X, y=None):
    
        if self.strategy is 'most_frequent':
            self.fill = {col: X[col].value_counts().index[0] for 
            col in self.columns}
    
        if self.columns is None:
            self.columns = X.columns
    
        else:
            self.fill ={col: '0' for col in self.columns}
      
        return self
      
    def transform(self,X):
        X_2 = X.copy()
        for col in self.columns:
            X_2[col] = X_2[col].fillna(self.fill[col])
        return X_2

Used pd.get_dummies to convert the categorical values to numerical values.

In [ ]:
class CategoricalEncoder(BaseEstimator, TransformerMixin):
  
    def __init__(self, dropFirst=True):
        self.categories = dict()
        self.dropFirst = dropFirst
    
    def fit(self, X, y=None):

        df_cat = df.select_dtypes(include=['object'])
        for col in df_cat.columns:
            self.categories[col] = df_cat[col].value_counts().index.tolist()
        return self
    
    def transform(self, X):
        X_2 = X.copy()
        X_2 = X_2.select_dtypes(include=['object'])
        for col in X_2.columns:
            X_2[col] = X_2[col].astype({col: CategoricalDtype(self.categories[col])})
            
        return pd.get_dummies(X_2, drop_first=self.dropFirst)

Complete Categorical Pipeline

In [ ]:
cat_pipeline = Pipeline(steps=[
    ("cat_selector", featureSelector(type='object')),
    ("cat_imputer", CategoricalImputer(columns = ['occupation', 'workClass', 'native_country'])),
    ("cat_encoder", CategoricalEncoder())
])

Split df to train data and test data

In [ ]:
Train_x = df[df['income'].notnull()].drop('income', axis=1)
Train_y = df[df['income'].notnull()]['income']
Test_x = df[df['income'].isnull()].drop('income', axis=1).reset_index(drop=True)
Test_y = Test_y

Completed Pipeline uses pipline

In [ ]:
# full_pipeline = FeatureUnion([("num_pipeline", num_pipeline), 
#                 ("cat_pipeline", cat_pipeline)])

Train_x_num = num_pipeline.fit_transform(Train_x)
Train_x_cat = cat_pipeline.fit_transform(Train_x)
Train = pd.concat([Train_x_num, Train_x_cat, Train_y], axis=1)

In [ ]:
Test_x_num = num_pipeline.fit_transform(Test_x)
Test_x_cat = cat_pipeline.fit_transform(Test_x)
Test = pd.concat([Test_x_num, Test_x_cat, Test_y], axis=1)

# -------------------------------------------model-----------------------------------------

In [ ]:
Train_df = Train
Test_df = Test

In [ ]:
Train_x = Train_df.drop('income', axis=1)
Train_y = Train_df['income']
Test_x = Test_df.drop('income', axis=1)
Test_y = Test_df['income']

Created the model

In [ ]:
from sklearn.model_selection import KFold

kf = KFold(n_splits=5, shuffle=True, random_state = 2021)
best_model = LogisticRegression(C=1, max_iter=10, penalty='l1', random_state=0, solver='liblinear')

predictions = np.zeros(len(Test_x))
acss = []

for i, (trn_idx, val_idx) in enumerate(kf.split(Train_x, Train_y)):
    print('--------------------- {} fold ---------------------'.format(i+1))
    trn_x, trn_y = Train_x.iloc[trn_idx], Train_y.iloc[trn_idx]
    val_x, val_y = Train_x.iloc[val_idx], Train_y.iloc[val_idx]
    
    best_model.fit(trn_x, trn_y)    
    pred_y = best_model.predict(val_x)
    
    acs = accuracy_score(val_y, pred_y)
    acss.append(acs)
    predictions += best_model.predict(Test_x) / kf.n_splits
    
    print('accuracy: ', acs)

print('average accuracy: ', np.mean(acss))

In [ ]:
predictions = predictions.astype('int')
accuracy_score(predictions, Test_y.values)

Save mode

In [ ]:
import pickle

filename = 'best_model.sav'
pickle.dump(best_model, open(filename, 'wb'))

In [ ]:
predictions.to_csv('prediction.csv')